# EEG Preprocessing Pipeline
This notebook demonstrates how we preprocessed the EEG data obtained from a single subject.
## Contents
Our preprocessing p

In [1]:
!pip install mne
!pip install numpy
!pip install pickle

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)

[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)

[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [12]:
import mne as mne
import numpy as np
import pickle

In [3]:
file_ec = r"./data/example_eeg/example_subj_EC_raw.fif.gz"
file_eo = r"./data/example_eeg/example_subj_EO_raw.fif.gz"

raw_ec = mne.io.read_raw(file_ec, preload=True, verbose=False)
raw_eo = mne.io.read_raw(file_eo, preload=True, verbose=False)

In [4]:
def set_montage_layout(raw):
    raw = raw.copy()
    montage = mne.channels.make_standard_montage("GSN-HydroCel-129")
    raw.set_eeg_reference(ref_channels=['Cz'], verbose=False)
    raw.drop_channels([ch for ch in raw.ch_names if ch not in montage.ch_names])
    raw.set_montage(montage)
    return raw

In [5]:
def filter_powerline_noise(raw):
    raw = raw.copy()
    raw = raw.filter(1, 70, verbose=False)
    raw = raw.notch_filter(freqs=60, verbose=False)
    return raw

In [6]:
eeg_ec_montage = set_montage_layout(raw_ec)
eeg_eo_montage = set_montage_layout(raw_eo)

eeg_ec_filtered = filter_powerline_noise(raw_ec)
eeg_eo_filtered = filter_powerline_noise(raw_eo)

In [7]:
freq_bands = {
    'delta': [0.5, 4],
    'theta': [4, 7],
    'alpha': [7, 13],
    'beta': [13, 30],
    'whole_spec': [0.5, 30]
}

In [8]:
from config.interpolation_maps import chan_map_R5, chan_map_R12, chan_map_R20

def interpolate_channels(raw, chan_map=chan_map_R12):
    chan_groups = {}
    for key in chan_map.keys():
        chans = [raw.info['ch_names'].index(x) for x in chan_map[key]]
        chan_groups[key] = chans
    raw = mne.channels.combine_channels(raw, groups=chan_groups, method='mean', verbose=False)
    return raw

In [9]:
def filter_freq_bands_of_interest(raw, fboi=freq_bands, interpolate_chan=True):
    eeg_freq_bands = dict()
    for key in fboi.keys():
        fb_range = fboi[key]
        raw_temp = raw.copy().filter(fb_range[0], fb_range[1], verbose=False)
        raw_temp = raw_temp.resample(sfreq=250)
        if interpolate_chan:
            raw_temp = interpolate_channels(raw_temp)
        eeg_freq_bands[key] = raw_temp
    return eeg_freq_bands

In [10]:
eeg_ec_freq_bands = filter_freq_bands_of_interest(eeg_ec_filtered)
eeg_eo_freq_bands = filter_freq_bands_of_interest(eeg_eo_filtered)

for key in eeg_ec_freq_bands.keys():
    print(f"EEG-EC filtered in {key}{freq_bands[key]} with dimensions {np.array(eeg_ec_freq_bands[key].get_data(units='uV')).shape}")
print()
for key in eeg_eo_freq_bands.keys():
    print(f"EEG-EO filtered in {key}{freq_bands[key]} with dimensions {np.array(eeg_eo_freq_bands[key].get_data(units='uV')).shape}")

EEG-EC filtered in delta[0.5, 4] with dimensions (12, 9992)
EEG-EC filtered in theta[4, 7] with dimensions (12, 9992)
EEG-EC filtered in alpha[7, 13] with dimensions (12, 9992)
EEG-EC filtered in beta[13, 30] with dimensions (12, 9992)
EEG-EC filtered in whole_spec[0.5, 30] with dimensions (12, 9992)

EEG-EO filtered in delta[0.5, 4] with dimensions (12, 4984)
EEG-EO filtered in theta[4, 7] with dimensions (12, 4984)
EEG-EO filtered in alpha[7, 13] with dimensions (12, 4984)
EEG-EO filtered in beta[13, 30] with dimensions (12, 4984)
EEG-EO filtered in whole_spec[0.5, 30] with dimensions (12, 4984)


In [14]:
def save_object(obj, fname):
    try:
        with open(fname + ".pickle", "wb") as f:
            pickle.dump(obj, f, protocol=pickle.HIGHEST_PROTOCOL)
    except Exception as ex:
        print("Error during pickling object (Possibly unsupported):", ex)

save_object(eeg_ec_freq_bands, r"data/example_eeg/example_subj_EC_preproc")
save_object(eeg_eo_freq_bands, r"data/example_eeg/example_subj_EO_preproc")